<a href="https://colab.research.google.com/github/minhnhat232k/Bigdata/blob/main/Assignment2_Bigdata_q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.0.1

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
spark = SparkSession.builder.getOrCreate()
ratingsDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/ratings_small.csv")
moviesDF = spark.read.options(header='True', inferSchema='True', delimiter=',').csv("/content/gdrive/MyDrive/Bigdata-Pyspark-Dataset/Assignment2/movies_small.csv")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:36635)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Py4JNetworkError: ignored

In [ ]:
moviesDF = moviesDF.withColumn("genres",f.explode(f.split("genres", "\\|")))
moviesDF.show()

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, FloatType


# Step 1: Calculate average rating by genre and by movieId
genre_ratings = ratingsDF.join(moviesDF, on='movieId', how='inner').groupBy('genres') \
    .agg(f.avg('rating').alias('genre_avg_rating'), f.collect_list('movieId').alias('genre_movies'))

movie_ratings = ratingsDF.groupBy('movieId').agg(f.avg('rating').alias('movie_avg_rating'))

# Step 2: Find highest rated genre for each user
user_highest_rated_genre = ratingsDF.join(moviesDF, on='movieId', how='inner').groupBy('userId', 'genres') \
    .agg(f.avg('rating').alias('user_genre_avg_rating')) \
    .withColumn('rank', f.rank().over(Window.partitionBy('userId').orderBy(f.desc('user_genre_avg_rating')))) \
    .where(f.col('rank') == 1) \
    .select('userId', 'genres', 'user_genre_avg_rating')

# Step 3: Find top 5 unrated movies with highest rating for each highest rated genre
# Create a DataFrame containing 'movieId' and 'genres'
movies_id_genre = ratingsDF.join(moviesDF, on='movieId', how='inner').select('movieId', 'genres')

# Join 'movie_ratings' with DataFrame to get access to 'movie_avg_rating'
unrated_movies_with_ratings = movies_id_genre.join(movie_ratings, on='movieId', how='inner')

# Define a UDF to extract values from an array column
extract_rating_udf = f.udf(lambda arr: [item['rating'] for item in arr], ArrayType(FloatType()))

# Calculate the top 5 unrated movies for each user and genre
top_unrated_movies = user_highest_rated_genre.join(genre_ratings, on='genres', how='inner') \
    .join(unrated_movies_with_ratings, on='genres', how='inner') \
    .withColumn('unrated_movies', f.array_except('genre_movies', f.collect_list('movieId').over(Window.partitionBy('userId')))) \
    .withColumn('unrated_movie_avg_rating', extract_rating_udf('unrated_movies')) \
    .withColumn('top_5_unrated_movies', f.explode(f.arrays_zip(f.col('unrated_movies'), f.col('unrated_movie_avg_rating')))) \
    .select('userId', 'genres', 'genre_avg_rating', 'top_5_unrated_movies.*') \
    .orderBy('userId', 'genres', f.desc('unrated_movie_avg_rating')) \
    .groupBy('userId', 'genres', 'genre_avg_rating') \
    .agg(f.collect_list(f.struct('unrated_movies', 'unrated_movie_avg_rating')).alias('top_5_unrated_movies'))
top_unrated_movies.show(truncate=False)
